#Cargo las bibliotecas

In [1]:
import math
import pandas as pd

#Cargo los archivos

In [2]:
parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/train.parq'

train_data = pd.read_parquet(parquet_file)

train_data

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:exploit,0,6466,0
1,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:spam,0,6466,0
2,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:bruteforce,2,4637,0
3,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:spam,2,4637,0
4,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:exploit,2,4637,0
...,...,...,...,...,...,...,...,...,...,...,...
61629680,2023-08-24 21:14:52+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3209.0,Vodafone GmbH,http:scan,24879,190784,0
61629681,2023-08-24 21:14:46+00:00,US,22612.0,NAMECHEAP-NET,US,23033.0,WOW,http:scan,40383,193446,0
61629682,2023-08-24 21:14:46+00:00,US,22612.0,NAMECHEAP-NET,US,23033.0,WOW,http:spam,40383,193446,0
61629683,2023-08-24 21:14:46+00:00,US,22612.0,NAMECHEAP-NET,US,23033.0,WOW,http:spam,40383,193446,0


#Data_set

Eliminacion de duplicados

In [3]:
batch_size = 1000000 #Un millón
num_batches = len(train_data) // batch_size #Establece la cantidad de batches que voy a necesitar

duplicated_data = []

for i in range(num_batches):
    batch = train_data[i * batch_size : (i + 1) * batch_size] #Establece de donde hasta donde voy a tomar los valores
    duplicated_batch = batch.drop_duplicates() #Dropea los duplicados dentro de esos valores
    duplicated_data.append(duplicated_batch)

train_data = pd.concat(duplicated_data)
del duplicated_batch
del batch

#Imputacion de valores nulos

Oberservamos que hay valores Nan en ciertas columnas de nustro data_set y consideramos que en el caso de lo "country", ese Nan representa que no se conoce el pais, mientras que en los num se interpreto como que ese atacante no tiene identificacion. Por estos, luego de analizar la cantidad de valores que tiene cada elemento, se decidió cederles estos nulos a los valores que eran mas representativos con diferencia de cada columna.

Con representativos nos referimos a que, por ejemplo, el pais US tiene una cantidad de apariciones de 8 cifras, y el segundo lugar solo 7, por lo que se optó por agregar los valores nulos de attacker_country a "US" ya que esto no marcaba una diferencia considerable en la propocion del valor de la misma categoria

In [4]:
listaVariables = train_data.count() #Concluyo que watcher_country, watcher_as_name, attacker_as_name, attacker_country y attacker_as_num tienen valores Nan
variables = [listaVariables.index[1], listaVariables.index[3], listaVariables.index[4], listaVariables.index[5], listaVariables.index[6]]

for i in range(5):
  values = train_data[variables[i]].value_counts()
  first_value = values.index[0]
  train_data[variables[i]] = train_data[variables[i]].fillna(first_value)

del listaVariables

In [5]:
train_data.count() #Se reemplazaron correctamente.

attack_time          40041188
watcher_country      40041188
watcher_as_num       40041188
watcher_as_name      40041188
attacker_country     40041188
attacker_as_num      40041188
attacker_as_name     40041188
attack_type          40041188
watcher_uuid_enum    40041188
attacker_ip_enum     40041188
label                40041188
dtype: int64

#Agrupacion de categorias con bajo valor

In [6]:
listaVariables = train_data.columns.tolist()
variables1 = [listaVariables[1],  listaVariables[4]] #watcher_country, attacker_country

In [7]:
for i in range(2):
  #Se tomó 1.000 como referencia en todos ya que, despues de comprobar los valores, por debajo de eso se consideró insignificante respecto a los demas.
  value_counts = train_data[variables1[i]].value_counts()

  # Encuentra los valores que aparecen menos de 1000 veces
  valores_menos_recuentes = value_counts[value_counts < 1000].index

  # Sustituye estos valores por 'otros'
  train_data[variables1[i]] = train_data[variables1[i]].cat.add_categories('otros')
  train_data.loc[train_data[variables1[i]].isin(valores_menos_recuentes), variables1[i]] = 'otros'

  #Elimina las categorias que no tienen ninguna valor
  train_data[variables1[i]].cat.remove_unused_categories(inplace=True)

del variables1
del value_counts
del valores_menos_recuentes

<ipython-input-7-344d573c3236>:13: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data[variables1[i]].cat.remove_unused_categories(inplace=True)
<ipython-input-7-344d573c3236>:13: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data[variables1[i]].cat.remove_unused_categories(inplace=True)


In [11]:
del listaVariables

#Creo nuevas features

mean_per_enum_category

In [12]:
mean_per_enum_category = train_data.groupby(['attacker_as_name', 'attacker_country'])['attacker_ip_enum'].mean().reset_index(name='mean_per_ipenum')
train_data = train_data.merge(mean_per_enum_category, on=['attacker_as_name', 'attacker_country'], how='left')
del mean_per_enum_category

count_per_enum_category

In [13]:
count_per_enum_category = train_data.groupby(['attacker_as_name', 'attacker_country'])['attacker_ip_enum'].count().reset_index(name='count_per_ipenum')
train_data = train_data.merge(count_per_enum_category, on=['attacker_as_name', 'attacker_country'], how='left')
del count_per_enum_category

attacker_per_country

In [14]:
attackerip_per_country = train_data.groupby(['attacker_country'])['attacker_ip_enum'].mean()
train_data['attackerip_per_country']= train_data['attacker_country'].map(attackerip_per_country).astype('int')
del attackerip_per_country

attackerip_per_name

In [15]:
attackerip_per_name = train_data.groupby(['attacker_as_name'])['attacker_ip_enum'].mean()
train_data['attackerip_per_name']= train_data['attacker_as_name'].map(attackerip_per_name).astype('int')
del attackerip_per_name

attackerip_per_time

In [16]:
attackerip_per_time = train_data.groupby(['attack_time'])['attacker_ip_enum'].mean()
train_data['attackerip_per_time']= train_data['attack_time'].map(attackerip_per_time).astype('int')
del attackerip_per_time

Ordenamos el label

In [17]:
# Muevo la columna 'label' al final
train_data = train_data[[col for col in train_data.columns if col != 'label'] + ['label']]

#Algunas transformaciones y drops

Nos interesa poder manejar el tiempo en un formato que sea mas amigable y facil de seguir por lo que nos quedaremos con las horas unicamente ya que todos los años son iguales, y los meses solo tienen dos posibles valores. Los segundos y minutos tambien consideré que carecia de valor

In [18]:
train_data['attack_time'] = train_data['attack_time'].dt.hour + train_data['attack_time'].dt.minute / 60.0

Debido a que attacker_as_name y watcher_as_name representan lo mismo que su contraparte de attacker_as_num y watcher_as_num, en un principio decidiremos quedarnos solo con los que eventualmente funcionen mejor para nuestro modelo

In [19]:
train_data = train_data.drop(columns=['watcher_as_num','attacker_as_num'])

#Casteos de tipos de datos

In [20]:
train_data

,attack_time,watcher_country,watcher_as_name,attacker_country,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,mean_per_ipenum,count_per_ipenum,attackerip_per_country,attackerip_per_name,attackerip_per_time,label
0,7.283333,DE,Host Europe GmbH,TR,Murat Aktas,http:exploit,0,6466,6470.080585,121065,60279,6470,90401,0
1,7.283333,DE,Host Europe GmbH,TR,Murat Aktas,http:spam,0,6466,6470.080585,121065,60279,6470,90401,0
2,7.283333,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:bruteforce,2,4637,84007.011771,537782,95007,83712,85982,0
3,7.283333,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:spam,2,4637,84007.011771,537782,95007,83712,85982,0
4,7.283333,DE,bn:t Blatzheim Networks Telecom GmbH,DE,Contabo GmbH,http:exploit,2,4637,84007.011771,537782,95007,83712,85982,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40041183,10.683333,US,LIQUIDWEB,US,UNIFIEDLAYER-AS-1,http:scan,14494,196833,95940.124103,255666,97204,96108,132991,0
40041184,10.683333,DE,Host Europe GmbH,SG,AMAZON-02,http:scan,5578,190687,95698.728470,564852,107871,95121,128542,0
40041185,10.683333,DE,Host Europe GmbH,SG,AMAZON-02,http:spam,5578,190687,95698.728470,564852,107871,95121,128542,0
40041186,10.683333,AU,Aussie Broadband,AU,Host Universal Pty Ltd,tcp:scan,17620,193153,179502.250614,10171,103948,163453,102139,0


In [21]:
train_data['attack_time'] = pd.to_numeric(train_data['attack_time'], downcast='float')
train_data['watcher_uuid_enum'] = pd.to_numeric(train_data['watcher_uuid_enum'], downcast='integer')
train_data['attacker_ip_enum'] = pd.to_numeric(train_data['attacker_ip_enum'], downcast='integer')
train_data['mean_per_ipenum'] = pd.to_numeric(train_data['mean_per_ipenum'], downcast='float')
train_data['count_per_ipenum'] = pd.to_numeric(train_data['count_per_ipenum'], downcast='integer')
train_data['attackerip_per_country'] = pd.to_numeric(train_data['attackerip_per_country'], downcast='integer')
train_data['attackerip_per_name'] = pd.to_numeric(train_data['attackerip_per_name'], downcast='integer')
train_data['attackerip_per_time'] = pd.to_numeric(train_data['attackerip_per_time'], downcast='integer')

#Almaceno el dataset analizado

In [22]:
train_data.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/XGBoost (Modelo1)/train_set_modelos_analizado.parq')

In [23]:
del train_data

#Cargo los datos de Kaggle

In [24]:
parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/test.parq'

test_data = pd.read_parquet(parquet_file)

test_data

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum
0,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:spam,1,7696
1,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:exploit,1,7696
2,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
3,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
4,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:bruteforce,3,7543
...,...,...,...,...,...,...,...,...,...,...
18682292,2023-08-24 21:14:42+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081
18682293,2023-08-24 21:14:52+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081
18682294,2023-08-24 21:14:42+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081
18682295,2023-08-24 21:14:42+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081


#Kaggle_Set

In [25]:
listaVariables = test_data.count() #Concluyo que watcher_country, watcher_as_name, attacker_country, attacker_as_num y attacker_as_name tienen valores Nan
variables = [listaVariables.index[1], listaVariables.index[3], listaVariables.index[4], listaVariables.index[5], listaVariables.index[6]]

for i in range(5):
  values = test_data[variables[i]].value_counts()
  first_value = values.index[0]
  test_data[variables[i]] = test_data[variables[i]].fillna(first_value)

del listaVariables

#Creo nuevas features

mean_per_enum_category

In [26]:
mean_per_enum_category = test_data.groupby(['attacker_as_name', 'attacker_country'])['attacker_ip_enum'].mean().reset_index(name='mean_per_ipenum')
test_data = test_data.merge(mean_per_enum_category, on=['attacker_as_name', 'attacker_country'], how='left')
del mean_per_enum_category

count_per_enum_category

In [27]:
count_per_enum_category = test_data.groupby(['attacker_as_name', 'attacker_country'])['attacker_ip_enum'].count().reset_index(name='count_per_ipenum')
test_data = test_data.merge(count_per_enum_category, on=['attacker_as_name', 'attacker_country'], how='left')
del count_per_enum_category

attacker_per_country

In [28]:
attackerip_per_country = test_data.groupby(['attacker_country'])['attacker_ip_enum'].mean()
test_data['attackerip_per_country']= test_data['attacker_country'].map(attackerip_per_country).astype('int')
del attackerip_per_country

attackerip_per_name

In [29]:
attackerip_per_name = test_data.groupby(['attacker_as_name'])['attacker_ip_enum'].mean()
test_data['attackerip_per_name']= test_data['attacker_as_name'].map(attackerip_per_name).astype('int')
del attackerip_per_name

attackerip_per_time

In [30]:
attackerip_per_time = test_data.groupby(['attack_time'])['attacker_ip_enum'].mean()
test_data['attackerip_per_time']= test_data['attack_time'].map(attackerip_per_time).astype('int')
del attackerip_per_time

#Repito las operaciones del data_set anterior

In [31]:
test_data['attack_time'] = test_data['attack_time'].dt.hour + test_data['attack_time'].dt.minute / 60.0

In [32]:
test_data = test_data.drop(columns=['watcher_as_num','attacker_as_num'])

In [33]:
test_data['attack_time'] = pd.to_numeric(test_data['attack_time'], downcast='float')
test_data['watcher_uuid_enum'] = pd.to_numeric(test_data['watcher_uuid_enum'], downcast='integer')
test_data['attacker_ip_enum'] = pd.to_numeric(test_data['attacker_ip_enum'], downcast='integer')
test_data['mean_per_ipenum'] = pd.to_numeric(test_data['mean_per_ipenum'], downcast='float')
test_data['count_per_ipenum'] = pd.to_numeric(test_data['count_per_ipenum'], downcast='integer')
test_data['attackerip_per_country'] = pd.to_numeric(test_data['attackerip_per_country'], downcast='integer')
test_data['attackerip_per_name'] = pd.to_numeric(test_data['attackerip_per_name'], downcast='integer')
test_data['attackerip_per_time'] = pd.to_numeric(test_data['attackerip_per_time'], downcast='integer')

#Almaceno el Kaggle_set analizado

In [34]:
test_data.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/XGBoost (Modelo1)/test_set_modelos_analizado.parq')